In [2]:
import os
os.chdir("/Users/travishong/aml-lsc-hierarchy")  # set once

import pandas as pd

df = pd.read_csv("data/Fig3_Relapse_Deconvolution/scArches_PVG_Abbas_CellType_Classified.csv")
print(df.shape)
print(df.columns)
df.head()

(33841, 6)
Index(['Cell', 'Patient', 'Cohort', 'UMAP1', 'UMAP2', 'CellType'], dtype='object')


,Cell,Patient,Cohort,UMAP1,UMAP2,CellType
0,AML328-D0_AAAACCGCTACT-0,AML328,vanGalen,12.895105,0.464960,LSPC-Quiescent
1,AML328-D0_AAAACCGGCTTT-0,AML328,vanGalen,13.200820,1.895669,LSPC-Quiescent
2,AML328-D0_AAAAGTCCCCGT-0,AML328,vanGalen,0.019062,4.923256,Mono-like
3,AML328-D0_AAAAGTGCCAAT-0,AML328,vanGalen,1.899196,2.733148,cDC-like
4,AML328-D0_AAAAGTTGGCGT-0,AML328,vanGalen,0.489852,3.284149,Mono-like


In [3]:
import numpy as np

cols = [c.lower() for c in df.columns]

# 1) Patient ID
pat_col = next(c for c in df.columns if "patient" in c.lower() or "case" in c.lower())

# 2) Timepoint / sample type
tp_candidates = [c for c in df.columns if any(k in c.lower() for k in ["timepoint", "sampletype", "visit", "status"])]
tp_col = tp_candidates[0]

# 3) Outcome / relapse info
out_candidates = [c for c in df.columns if any(k in c.lower() for k in ["relapse", "outcome", "rfs", "group"])]
out_col = out_candidates[0]

# 4) Predicted hierarchy / cell type from scArches
hier_candidates = [c for c in df.columns if any(k in c.lower() for k in ["celltype", "hierarchy", "class", "linclass"])]
# pick the *most specific* one; if there are multiple, you can print them and choose
print("patient:", pat_col)
print("timepoint-like:", tp_candidates)
print("outcome-like:", out_candidates)
print("hierarchy-like:", hier_candidates)


IndexError: list index out of range

In [4]:
from src.hierarchy_classifier import map_zeng_celltype_to_hierarchy

df["hierarchy"] = df["CellType"].map(map_zeng_celltype_to_hierarchy)
df["hierarchy"].value_counts()

hierarchy
Primitive     24219
Mature         5383
Progenitor     4239
Name: count, dtype: int64

In [5]:
abbas = df[df["Cohort"] != "vanGalen"].copy()

import numpy as np

rows = []
for pid, sub in abbas.groupby("Patient"):
    total = sub.shape[0]
    if total < 50:
        continue

    freq = sub["hierarchy"].value_counts(normalize=True)

    rows.append({
        "patient_id": pid,
        "n_cells": total,
        "frac_Primitive": freq.get("Primitive", 0.0),
        "frac_Progenitor": freq.get("Progenitor", 0.0),
        "frac_Mature": freq.get("Mature", 0.0),
    })

abbas_comp = pd.DataFrame(rows).sort_values("frac_Primitive", ascending=False)
abbas_comp.head()


,patient_id,n_cells,frac_Primitive,frac_Progenitor,frac_Mature
7,PT8A,956,0.993724,0.001046,0.005230
4,PT5A,5171,0.989170,0.005221,0.005608
0,PT1A,5030,0.945924,0.007555,0.046521
5,PT6A,2476,0.926898,0.042811,0.030291
6,PT7A,2677,0.912589,0.052671,0.034740


In [6]:
abbas_comp.to_csv("data/abbas_patient_hierarchy_composition_from_scarches.csv", index=False)

In [16]:
from src.hierarchy_classifier import load_hierarchy_model

clf, ref_genes = load_hierarchy_model()